
# 🌐 Hugging Face + 다국어 번역 & TTS 서비스 실습 노트북

이 노트북에서는 다음 내용을 다룹니다.

1. Hugging Face 개념 정리
2. 프로젝트에 사용된 모델 개요 (M2M100, Kokoro, MMS-TTS-KOR, Uroman)
3. 필수 라이브러리 설치
4. 번역 + TTS(영어/한국어) 통합 코드
5. Gradio 웹 UI로 서비스 실행

---



## 0. Hugging Face란 무엇인가?
https://huggingface.co/

Hugging Face는 **AI 모델 공유·배포 플랫폼**이자 **🤗 Transformers 라이브러리**의 개발사입니다.

### 0-1. Hugging Face Hub

- GitHub처럼 **AI 모델, 데이터셋, Space(앱)**를 공유하는 공간
- 모델을 Git처럼 push/pull 할 수 있음
- 예시 모델들
  - `facebook/m2m100_418M` (다국어 번역)
  - `facebook/mms-tts-kor` (한국어 TTS)
  - `openai/whisper-large-v3` (음성 인식)
  - `meta-llama/Llama-3-8B` (LLM)

### 0-2. Transformers 라이브러리

Transformers 라이브러리는 다양한 딥러닝 모델을 **아주 간단한 코드**로 불러와 사용할 수 있게 해줍니다.

```python
from transformers import AutoModel, AutoTokenizer

model = AutoModel.from_pretrained("facebook/m2m100_418M")
tokenizer = AutoTokenizer.from_pretrained("facebook/m2m100_418M")
```

### 0-3. Pipeline API

작업(task)에 맞는 파이프라인을 선택하면, 세부 구성 없이 바로 사용 가능합니다.

```python
from transformers import pipeline

translator = pipeline("translation", model="facebook/m2m100_418M")
translator("안녕하세요")
```

### 0-4. Spaces와 Gradio

- **Spaces**: Hugging Face에서 Gradio/Streamlit 앱을 무료로 호스팅할 수 있는 공간
- 이 노트북에서 만드는 번역+TTS 서비스를 그대로 Spaces에 올릴 수 있습니다.



## 1. 이 프로젝트에 사용된 모델 개요

이번 프로젝트는 HF 모델 3개를 결합한 **멀티 모달 번역-TTS 시스템**입니다.

### 1-1. M2M100 (facebook/m2m100_418M)

- Meta가 만든 **100개 언어 ↔ 100개 언어 직접 번역** 모델
- 영어를 거치지 않고 언어 간 직접 번역 가능
- `src_lang`, `tgt_lang`으로 언어 제어

### 1-2. Kokoro-82M (영어 TTS)

- 고품질 영어 음성 합성 모델
- `kokoro.KPipeline`으로 쉽게 사용
- 여러 목소리(voice)를 선택 가능 (예: `af_heart`, `af_alloy` 등)

### 1-3. MMS-TTS-KOR (facebook/mms-tts-kor)

- Meta의 MMS 프로젝트에 포함된 **한국어 TTS** 모델
- 입력은 **로마자 문자열**(Latin 문자)을 기대함
- 따라서 한국어 문장은 먼저 Uroman으로 로마자로 변환해야 함

### 1-4. Uroman

- 전 세계 언어를 **로마자(라틴 문자)**로 변환하는 라이브러리
- `pip install uroman` 으로 설치 가능
- MMS-TTS-KOR에서 한국어를 처리하기 위한 필수 전처리 도구



## 2. 환경 준비 (필요 라이브러리 설치)

아래 코드는 이 노트북에서 사용할 주요 라이브러리를 설치하는 예시입니다.

> ⚠️ `torch`(PyTorch)는 GPU/OS에 따라 설치 명령이 달라서, 여기서는 별도로 안내만 합니다.


In [ ]:

# 라이브러리 설치 (필요 시 주석 해제 후 실행)
# !pip install --upgrade gradio numpy soundfile
# !pip install --upgrade transformers huggingface-hub sentencepiece accelerate
# !pip install kokoro uroman

# PyTorch는 https://pytorch.org/get-started/locally/ 에서
# 본인 환경(CUDA/CPU/OS)에 맞는 설치 명령을 복사해서 사용하세요.



## 3. 전체 시스템 아키텍처

1. 사용자가 텍스트와 **원본 언어 / 번역 언어**를 선택해서 입력
2. `translate_and_speak()` 함수가:
   - M2M100으로 번역
   - 목표 언어가 영어면 Kokoro, 한국어면 MMS-TTS-KOR로 TTS 실행
3. 생성된 텍스트 + 음성 파일 경로를 Gradio 컴포넌트에 전달
4. 브라우저에서 번역 결과와 플레이어로 음성을 재생

```text
입력 텍스트
      ↓
[원본/타겟 언어 선택]
      ↓
M2M100 번역 (facebook/m2m100_418M)
      ↓
번역된 텍스트
      ↓
TTS 분기
   ├─ 타깃=English → Kokoro TTS
   └─ 타깃=Korean  → Uroman → MMS-TTS-KOR
      ↓
Waveform(WAV) 생성
      ↓
Gradio Audio Player로 재생
```



## 4. 번역 + TTS 통합 코드

아래 코드는:

- M2M100 번역 모델
- Kokoro 영어 TTS
- MMS-TTS-KOR 한국어 TTS(+Uroman)
- Gradio UI

를 모두 합친 완성 코드입니다.


In [ ]:

import gradio as gr
import torch
import soundfile as sf
import numpy as np

from transformers import (
    M2M100ForConditionalGeneration,
    M2M100Tokenizer,
    VitsModel,
    AutoTokenizer,
)
from kokoro import KPipeline   # Kokoro-82M 파이썬 래퍼
from uroman import Uroman      # pip install uroman

# -----------------------------
# 0. 디바이스 (GPU 우선)
# -----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# -----------------------------
# 1. 번역 모델 (M2M100)
# -----------------------------
translation_model_name = "facebook/m2m100_418M"
translation_model = M2M100ForConditionalGeneration.from_pretrained(
    translation_model_name
).to(device)
translation_tokenizer = M2M100Tokenizer.from_pretrained(translation_model_name)

# -----------------------------
# 2. 영어 TTS: Kokoro-82M
# -----------------------------
kokoro_pipeline = KPipeline(lang_code="a")

def tts_english_kokoro(text: str, path: str = "tts_en.wav") -> str | None:
    text = (text or "").strip()
    if not text:
        return None

    generator = kokoro_pipeline(text, voice="af_heart")

    chunks = []
    for _, _, audio in generator:
        chunks.append(audio)

    if not chunks:
        return None

    audio_cat = np.concatenate(chunks)
    sf.write(path, audio_cat, 24000)
    return path

# -----------------------------
# 3. 한국어 TTS: MMS-TTS-KOR (VITS) + Uroman
# -----------------------------
kor_tts_name = "facebook/mms-tts-kor"
kor_tts_model = VitsModel.from_pretrained(kor_tts_name).to(device)
kor_tts_tokenizer = AutoTokenizer.from_pretrained(kor_tts_name)

uroman = Uroman()   # 한 번만 초기화

def tts_korean_mms(text: str, path: str = "tts_ko.wav") -> str | None:
    text = (text or "").strip()
    if not text:
        return None

    # 1) Uroman으로 로마자 변환
    roman_text = uroman.romanize_string(text)

    # 2) 토크나이저 입력 생성
    inputs = kor_tts_tokenizer(roman_text, return_tensors="pt").to(device)

    # 3) 모델 인퍼런스
    with torch.no_grad():
        outputs = kor_tts_model(**inputs).waveform

    wav = outputs.squeeze().cpu().numpy()
    if wav.size == 0:
        return None

    # 4) WAV 파일로 저장
    sf.write(path, wav, samplerate=kor_tts_model.config.sampling_rate)
    return path

# -----------------------------
# 4. 언어 매핑
# -----------------------------
LANGUAGES = {
    "Korean": "ko",
    "English": "en",
    "Chinese": "zh",
    "Japanese": "ja",
    "French": "fr",
    "Spanish": "es",
    "German": "de",
    "Hindi": "hi",
}

# -----------------------------
# 5. 번역 + TTS 함수
# -----------------------------
def translate_and_speak(text, src_lang_name, tgt_lang_name):
    text = text or ""
    if not text.strip():
        return "", None

    src_lang = LANGUAGES[src_lang_name]
    tgt_lang = LANGUAGES[tgt_lang_name]

    # ----- 번역 -----
    if src_lang == tgt_lang:
        translated_text = text.strip()
    else:
        translation_tokenizer.src_lang = src_lang
        encoded = translation_tokenizer(
            text,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=512,
        ).to(device)

        with torch.no_grad():
            generated_tokens = translation_model.generate(
                **encoded,
                forced_bos_token_id=translation_tokenizer.get_lang_id(tgt_lang),
                max_length=512,
            )

        translated_text = translation_tokenizer.batch_decode(
            generated_tokens, skip_special_tokens=True
        )[0]

    # ----- TTS -----
    audio_path = None
    if tgt_lang == "en":
        audio_path = tts_english_kokoro(translated_text)
    elif tgt_lang == "ko":
        audio_path = tts_korean_mms(translated_text)

    return translated_text, audio_path

# -----------------------------
# 6. Gradio UI
# -----------------------------
with gr.Blocks() as demo:
    gr.Markdown("## 🌐 다국어 번역 + TTS (영어 Kokoro + 한국어 MMS)")

    with gr.Row():
        src_lang = gr.Dropdown(
            list(LANGUAGES.keys()), label="원본 언어", value="Korean"
        )
        tgt_lang = gr.Dropdown(
            list(LANGUAGES.keys()), label="번역 언어", value="English"
        )

    with gr.Row():
        input_text = gr.Textbox(lines=8, label="입력 텍스트")
        output_text = gr.Textbox(lines=8, label="번역 결과", interactive=False)

    audio_output = gr.Audio(label="음성 출력", type="filepath")

    translate_button = gr.Button("번역 + 음성 생성")
    translate_button.click(
        fn=translate_and_speak,
        inputs=[input_text, src_lang, tgt_lang],
        outputs=[output_text, audio_output],
    )

    demo.queue()

# 주피터 노트북에서는 demo.launch()를 마지막 셀에서 직접 호출하는 것을 권장합니다.
print("모델 로딩 및 Gradio 앱 정의 완료. 다음 셀에서 demo.launch()를 실행하세요.")



## 5. Gradio 앱 실행

아래 셀을 실행하면 로컬 환경에서 Gradio 웹앱이 실행됩니다.

- 출력에 뜨는 링크(예: `http://127.0.0.1:7860`)를 클릭하면 브라우저에서 접속 가능합니다.


In [ ]:

# Gradio 앱 실행
demo.launch()
